In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr
from osgeo import gdal
import glob as glob
import os
import ntpath
%matplotlib inline

## Crop the index raster to the basins

In [2]:
sfs = glob.glob('/Volumes/data/basin_dataset_public/shapefiles/*/*.shp') # get the paths to all the shapefiles

In [3]:
driver = ogr.GetDriverByName('ESRI Shapefile')

In [4]:
inpth = '/Volumes/data/Theo/projects/Budyko_vic/data/vic_index_new_small.tif'

In [5]:
for sf in sfs: # loop through shapefiles
    data = driver.Open(sf)
    layer = data.GetLayer()

    features = []
    for feature in layer:
        features.append(feature.GetFieldAsString("GAGEID")) # pull out all the gage IDs

    features = np.unique(features) # create a list of unique features in the layer

    for feature in features: # look through and crop the index raster by each basin
        cmd = 'gdalwarp -t_srs "+proj=longlat +datum=WGS84 +no_defs" -cutline '+sf+' -cl '+layer.GetName()+""" -cwhere "GAGEID = '"""+feature+"""'" -crop_to_cutline """+inpth+' /Volumes/data/basin_dataset_public/index_rasters/'+layer.GetName()+'_'+feature+'.tif'
        os.system(cmd) # call the command on the system terminal

## Process the new basin index rasters

### Produce a list as such

- Basin 12345678 Cell 1567 Percent 0.02
- Basin 12345678 Cell 187635 Percent 1.0
- Basin 12345678 Cell 293747 Percent 0.85
- Basin 12345678 Cell 38723 Percent 0.42

In [6]:
rasters = glob.glob('/Volumes/data/basin_dataset_public/index_rasters/*.tif') # get paths to all the tifs just created

In [7]:
outcells = []
outpercents = []
outbasins = []

for rastpth in rasters:

    ds = gdal.Open(rastpth)
    rast = np.array(ds.GetRasterBand(1).ReadAsArray())
    rast[rast==-9999] = np.NaN

    # grab the basin gageID from the file path
    pth,basename = ntpath.split(rastpth)
    bsn = int(basename.split('_')[-1].split('.')[0])

    cells = np.unique(rast[np.isnan(rast)==False]) # find the number of VIC cells that contribute to the basin
    n = len(cells) # get the number of cells

    # compute the percent each cell contributes to the basin
    percents = []
    for cell in cells:
        percents.append(len(rast[rast==cell])/100.)

    outcells.extend(list(cells))
    outpercents.extend(list(percents))
    outbasins.extend(list(np.repeat(bsn,n)))# repeat the basin code so it is the same length as the number of cells

In [8]:
# save the output
np.savez_compressed('/Volumes/data/Theo/projects/Budyko_vic/data/VIC_cells_overlap.npz',cells=outcells,percents=outpercents,basins=outbasins)